# Generate conversation IDs for other EMIS issues

Produce a sample of 20 conversation IDs where:
- EMIS is the sending practice, and the failure is due to 'request not acknowledged'
- EMIS is the sending practice, and the failure is due to 'Large messages rejected due to timeout duration reached of overall transfer'


In [1]:
import pandas as pd
import numpy as np

In [2]:
transfers = pd.read_parquet("s3://prm-gp2gp-transfer-data-dev/14-day-cutoff/v4/2021/6/transfers.parquet")

In [3]:
emis_sending = transfers["sending_supplier"] == "EMIS"

## EMIS is the sending practice, and the failure is due to 'request not acknowledged'

In [4]:
request_not_acknowledged = transfers["failure_reason"] == "Request not Acknowledged"
transfers[emis_sending & request_not_acknowledged].sample(20)

,conversation_id,sla_duration,requesting_practice_asid,sending_practice_asid,requesting_supplier,sending_supplier,sender_error_codes,final_error_codes,intermediate_error_codes,status,failure_reason,date_requested,date_completed
221217,8BA7EE19-BC34-4660-84E6-45E910A36DAC,NaN,021552679017,200000000079,EMIS,EMIS,[],[],[],TECHNICAL_FAILURE,Request not Acknowledged,2021-06-21 08:48:47.824,NaT
7279,40AA4600-CA99-11EB-BE5C-E3FA2CFDBA35,NaN,185883527011,602079435049,SystmOne,EMIS,[],[],[],TECHNICAL_FAILURE,Request not Acknowledged,2021-06-11 09:41:53.352,NaT
144268,400DE54D-4140-4BC3-9A46-A5B14906B692,NaN,203877935049,232288836017,EMIS,EMIS,[],[],[],TECHNICAL_FAILURE,Request not Acknowledged,2021-06-28 11:49:22.360,NaT
132410,0E10E9B6-A029-4CAA-8F0D-11FEE54ADB82,NaN,359279756019,070082176010,EMIS,EMIS,[],[],[],TECHNICAL_FAILURE,Request not Acknowledged,2021-06-29 13:29:22.840,NaT
24824,9383F0E8-7BFE-4100-ADF6-16EDDDE157BA,NaN,202393302015,312418725015,EMIS,EMIS,[],[],[],TECHNICAL_FAILURE,Request not Acknowledged,2021-06-11 08:40:30.417,NaT
169774,C87B9F90-9D2E-43D1-81DD-02114C0F0FF5,NaN,642269836044,200000010867,EMIS,EMIS,[],[],[],TECHNICAL_FAILURE,Request not Acknowledged,2021-06-25 11:14:37.120,NaT
130187,6D3B2AB1-1724-4919-B533-906F7B028C08,NaN,200000000690,926102461049,EMIS,EMIS,[],[],[],TECHNICAL_FAILURE,Request not Acknowledged,2021-06-29 11:08:36.247,NaT
101540,EF203E70-C2E1-11EB-86A6-3B9554CCCB3B,NaN,200000001013,504100158048,SystmOne,EMIS,[],[],[],TECHNICAL_FAILURE,Request not Acknowledged,2021-06-01 14:02:00.417,NaT
222425,3C7566F0-D020-11EB-8466-8F1034FD739D,NaN,200000014825,987917953042,SystmOne,EMIS,[],[],[],TECHNICAL_FAILURE,Request not Acknowledged,2021-06-18 10:30:44.116,NaT
124719,3DBE6F54-80D5-4D19-B5BB-3A26FE9C64BF,NaN,662109573011,102743416012,EMIS,EMIS,[],[],[],TECHNICAL_FAILURE,Request not Acknowledged,2021-06-30 08:35:42.444,NaT


Looking at these example in the SPINE logs, most of these seem to be the cases where the only message in the converation was the initial request for GP2GP. (i.e the request was never responded to).

However, there seem to be a handful of other cases mixed in. The common theme for these cases is that the sender responded to the request but only after a very long time.
- `9383F0E8-7BFE-4100-ADF6-16EDDDE157BA` - did eventually do a full transfer, but request was only responded to *after about a month*
- `D89D0284-E042-486F-9C38-48620F37C2B2` - as above
- `C87B9F90-9D2E-43D1-81DD-02114C0F0FF5` - did eventually respond with an error code around a month later
- `338E9790-D341-11EB-ACC7-9B0E52FFDA17` - did eventually send an EHR and ack the request, but only after several weeks 
- `3FD20898-960C-492D-A328-C7DC26E38D32` - did eventually send an EHR and ack the request, but only after a couple of weeks

Finally this conversation was a bit difficult to classify:
- `16B6FB75-40EA-4E71-9D8C-42E0331E1BCF` - record was sent on time and looks to be integrated, but possibly interpreted incorrectly as request acknowledgement appeared to be missing from the SPINE logs

## EMIS is the sending practice, and the failure is due to transfer timeout

In [5]:
#Error code 25: Large messages rejected due to timeout duration reached of overall transfer
transfer_timeout = transfers["final_error_codes"].apply(lambda errors: 25 in errors)
transfers[emis_sending & transfer_timeout].sample(20)

,conversation_id,sla_duration,requesting_practice_asid,sending_practice_asid,requesting_supplier,sending_supplier,sender_error_codes,final_error_codes,intermediate_error_codes,status,failure_reason,date_requested,date_completed
185611,D4BF64A6-4D57-4E5E-AD02-DD17CBFD2203,678996.0,410747971019,847520267018,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-16 10:51:25.408,2021-06-24 07:28:47.906
20012,633A3CB6-BA1F-41D8-928D-D28F459787AF,1100105.0,152021647044,847520267018,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-10 15:55:17.727,2021-06-23 09:31:25.782
131254,E5E294EE-7F93-4C74-8AE0-B4ACA4E4E06E,29199.0,328003090045,073137245012,EMIS,EMIS,"[nan, nan]","[25.0, nan]",[],INTEGRATED_ON_TIME,None,2021-06-28 13:28:20.892,2021-07-05 13:39:00.556
107446,4AB02368-C3A8-491A-BF80-66D5916B6E70,238377.0,226386928049,200000023645,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-30 12:59:03.377,2021-07-03 07:12:41.102
155480,F793CB12-F42E-4F3C-B9FC-4BEA28579EBA,515025.0,200000010514,200000016639,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-22 07:57:32.135,2021-06-28 07:01:54.669
11898,E504669F-F8F0-4D8B-94DA-0A28C3D8AEA9,83352.0,338749174045,657805573042,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-10 08:07:54.368,2021-06-11 07:18:32.932
170572,925A2F8B-E037-45AC-B11F-D706A9CE325A,84333.0,200000023181,611122280047,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-23 07:43:56.747,2021-06-24 07:09:50.320
119302,46DB3D51-06D0-4938-A9C4-4344471F1373,66434.0,573960432041,847520267018,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-29 16:33:25.153,2021-06-30 11:00:58.089
52999,1B7DFC3B-61A8-43A1-A6D0-9B4E09770C9E,323344.0,005113801012,653855070047,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-02 13:31:31.286,2021-06-06 07:21:37.322
33849,CF26FA43-89AA-4D24-BEE9-08C2963BA830,163015.0,802589643013,847520267018,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-08 09:55:19.637,2021-06-10 07:12:39.769


Looking at the SPINE logs for these conversation, this sample presents an interesting cross section of issues.

There are threee failure patterns that appeared more than once.

Requestor sent a COPC continue but large fragment COPCs appear to have never been sent
- `D4BF64A6-4D57-4E5E-AD02-DD17CBFD2203`
- `633A3CB6-BA1F-41D8-928D-D28F459787AF`
- `46DB3D51-06D0-4938-A9C4-4344471F1373`
- `CF26FA43-89AA-4D24-BEE9-08C2963BA830`
- `82E4DEE5-A36A-46F6-A33B-DB767BEF48DA`
- `40E02F24-81AA-4305-8165-B7E79A02552A`
- `E504669F-F8F0-4D8B-94DA-0A28C3D8AEA9`
- `72AF37B9-4D6B-405A-ADA5-F86308835183` (EHR was resent a couple of weeks after initial request but stalled again in the same way)

Started sending, messages stop, then some days later the requestor reports a timeout. No more messages are sent.
- `4AB02368-C3A8-491A-BF80-66D5916B6E70`
- `F793CB12-F42E-4F3C-B9FC-4BEA28579EBA`
- `1B7DFC3B-61A8-43A1-A6D0-9B4E09770C9E`
- `763F58D6-AB76-4F8C-A4EE-9D6E6C91043C`
- `CCD1C6F9-198C-4BBE-8779-AE2DFAADDA13`
- `AB28FF0A-9FFD-4A2C-A9A4-A3607099A3D8`
- `6E5D186A-48D6-4806-8987-21AF72081948` (also had a duplicate ack of request by sender half way though transfer)

Transfer seemed to run into spine errors / large message re-assembly failure, several days later the requestor reports timeout.
- `99D0B87D-CC06-4F97-B0D5-56CCEBBB6F6C`
- `FAE4DC12-44DF-4FF0-8732-81E03164CF25`

There were also the following cases that were unique in our sample:  

`E5E294EE-7F93-4C74-8AE0-B4ACA4E4E06E`- Started sending, messages stop, around a week later the requestor reports a timeout. However, the sender then retransmits the EHR extract and the process appears to start again, running to completion. By the SLA definition this was interpreted as being an integration on time, but in reality the records was transferred after we expect the paper records to be printed.

`925A2F8B-E037-45AC-B11F-D706A9CE325A` - The EHR is sent but the requestor (also EMIS) appeared to not issue a COPC continue message… and then reports a timeout error about a day later. Then around several weeks later, the EHR is resent and this is integrated successfully (but again, this is very late - so we expect the paper to have been printed already)

`5741F734-7A60-49EF-ADE9-8C8ED4F6BABE` - transfer seems too stall with no error code, around a week later the EHR is resent multiple times, some of these are acknowledged as a duplicate by the requester, but the transfer appears to stall again and a couple of days later the requestor reports a timeout.

## EMIS is the sending practice, and the failure is due to transfer timeout (simpler examples)

In [6]:
single_positive_sender_ack = transfers["sender_error_codes"].apply(lambda errors: tuple(np.isnan(e) for e in errors)) == (True,)
only_transfer_timeout = transfers["final_error_codes"].apply(set) == {25}
no_intermediate_error_codes = transfers["intermediate_error_codes"].apply(len) == 0
transfers[emis_sending & only_transfer_timeout & no_intermediate_error_codes & single_positive_sender_ack].sample(20)

,conversation_id,sla_duration,requesting_practice_asid,sending_practice_asid,requesting_supplier,sending_supplier,sender_error_codes,final_error_codes,intermediate_error_codes,status,failure_reason,date_requested,date_completed
198731,52748F76-9795-4D49-A99A-D26656E348FC,60560.0,135776121042,847520267018,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-17 14:13:55.864,2021-06-18 07:03:38.189
108715,A026F50D-A8E1-427A-8AED-1135BD5C50CE,948331.0,599422982042,200000000126,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-29 15:09:40.812,2021-07-10 14:36:08.669
140445,2C14059D-5AD8-4290-81B8-588243533909,1006821.0,879065662048,609528239013,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-17 15:19:28.204,2021-06-29 07:01:44.767
212130,3EFC1CFB-69B1-4261-9E94-B762B61065C2,599667.0,394016123017,319346611043,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-15 08:28:33.234,2021-06-22 07:03:40.092
23843,40E02F24-81AA-4305-8165-B7E79A02552A,73284.0,200000023898,847520267018,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-09 10:42:12.820,2021-06-10 07:04:02.641
130490,59BEE041-5411-40E7-9476-FDC6F84521E5,268272.0,200000000400,886704986019,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-29 11:26:48.479,2021-07-02 13:58:34.993
166436,2C59F515-241E-4186-877B-963E6C2C91EE,52103.0,625428746048,886704986019,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-24 16:53:04.837,2021-06-25 07:21:50.095
122690,C4448200-F4C6-41C8-9DE4-32F298A89FC4,1205503.0,200000002338,834467264018,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-29 08:08:36.585,2021-07-13 07:01:42.034
100058,EBC14EDC-9BD2-40A0-87FF-009161B15237,71783.0,965172391041,847520267018,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-01 11:24:44.260,2021-06-02 07:21:27.659
136934,1AE5C374-A993-4E5E-82E1-DDA81A61157E,774677.0,434106228043,847520267018,EMIS,EMIS,[nan],[25.0],[],TECHNICAL_FAILURE,Final Error,2021-06-29 08:05:35.998,2021-07-08 07:18:37.856


## Appendix - error code 25 only appears in final error codes column

In [7]:
transfers["final_error_codes"].explode().value_counts()

15.0    16455
12.0     4402
31.0      681
11.0      619
99.0      316
25.0      277
30.0      123
28.0      121
17.0       95
20.0       20
21.0       17
9.0         1
Name: final_error_codes, dtype: int64

In [8]:
transfers["sender_error_codes"].explode().value_counts()

20.0    2451
30.0    1680
14.0    1012
10.0     617
6.0      209
23.0     114
19.0      67
24.0      55
7.0       12
99.0       8
Name: sender_error_codes, dtype: int64

In [9]:
transfers["intermediate_error_codes"].explode().value_counts()

29    972
Name: intermediate_error_codes, dtype: int64